In [1]:
import gdown

In [2]:
!gdown --id 1-3usuwhbiZUNWiZqUzRIJgnH1AjLCv0K 

/usr/local/lib/python3.11/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1-3usuwhbiZUNWiZqUzRIJgnH1AjLCv0K
To: /kaggle/working/Domain 1.feather
100%|██████████████████████████████████████| 97.4M/97.4M [00:06<00:00, 14.0MB/s]


In [3]:
!gdown --id 1HvmlbHuwrmRUYJSyP8fKzq8dqKKKPMsv

/usr/local/lib/python3.11/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1HvmlbHuwrmRUYJSyP8fKzq8dqKKKPMsv
To: /kaggle/working/Domain 2.feather
100%|██████████████████████████████████████| 84.1M/84.1M [00:02<00:00, 28.2MB/s]


In [4]:
!gdown --id 1rzQx4mugpWQmJbAQzDCiEgqivSp2u9b-

/usr/local/lib/python3.11/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1rzQx4mugpWQmJbAQzDCiEgqivSp2u9b-
To: /kaggle/working/Domain 3.feather
100%|██████████████████████████████████████| 65.4M/65.4M [00:02<00:00, 30.3MB/s]


In [42]:
import pandas as pd
import torch
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
from collections import OrderedDict
from typing import List, Dict
from sklearn.model_selection import train_test_split
import gc
import copy
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score

In [6]:
!pip install -q flwr[simulation]

import flwr as fl
from flwr.common import (
    EvaluateIns,
    EvaluateRes,
    FitIns,
    FitRes,
    Parameters,
    Scalar,
    Status,
    NDArrays,
    ndarrays_to_parameters,
    parameters_to_ndarrays,
)
import os

from flwr.server.client_manager import ClientManager
from flwr.server.client_proxy import ClientProxy
from flwr.server.strategy.aggregate import aggregate, weighted_loss_avg
import numpy as np
import pandas as pd
from typing import List, Tuple, Dict, Union, Optional
from functools import partial, reduce

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.7/66.7 MB 27.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 22.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.4/242.4 kB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.0/236.0 kB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.3/47.3 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 540.0/540.0 kB 33.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-api-core 1.34.1 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<4.0.0dev,>=3.19.5, but you have protobuf 4.25.7 which is incompatible.
google-spark-connect 0.5.2 requires google-api-core>=2.19.1, but you have google-api-core 1.34.1 which is incompatible.
google-cloud-bigtable 2.30.0 

2025-05-26 05:05:07.376225: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748235907.567410      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748235907.625264      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [43]:
def accuracy_fn(y_true, y_pred):
    correct = torch.eq(y_true, y_pred).sum().item()
    acc = (correct / len(y_pred)) * 100
    return acc

def get_parameters(net) -> List[np.ndarray]:
    return [val.cpu().numpy() for _, val in net.state_dict().items()]

def set_parameters(net, parameters: List[np.ndarray]):
    params_dict = zip(net.state_dict().keys(), parameters)
    state_dict = OrderedDict({k: torch.tensor(v) for k, v in params_dict})
    net.load_state_dict(state_dict)

from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score
import torch
import torch.nn as nn
import copy
import numpy as np

def train(net, trainloader, criterion, optimizer, device, proximal_mu: float = None):
    net.to(device)
    net.train()
    running_loss, running_corrects, tot = 0.0, 0, 0
    all_preds, all_labels = [], []

    global_params = copy.deepcopy(net).parameters()

    for images, labels in trainloader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = net(images)
        loss = criterion(outputs, labels)

        if proximal_mu is not None:
            proximal_term = sum((local - global_).norm(2)
                                for local, global_ in zip(net.parameters(), global_params))
            loss += (proximal_mu / 2) * proximal_term

        loss.backward()
        optimizer.step()

        preds = torch.argmax(outputs, dim=1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

        tot += images.size(0)
        running_corrects += torch.sum(preds == labels).item()
        running_loss += loss.item() * images.size(0)

    running_loss /= tot
    accuracy = running_corrects / tot

    cm = confusion_matrix(all_labels, all_preds)
    TP = np.diag(cm)
    FP = np.sum(cm, axis=0) - TP
    FN = np.sum(cm, axis=1) - TP
    TN = np.sum(cm) - (TP + FP + FN)

    precision = precision_score(all_labels, all_preds, average='macro', zero_division=0)
    recall = recall_score(all_labels, all_preds, average='macro', zero_division=0)
    f1 = f1_score(all_labels, all_preds, average='macro', zero_division=0)

    return running_loss, accuracy, precision, recall, f1, TP.sum(), FP.sum(), FN.sum(), TN.sum()



def test(net, testloader, device):
    net.to(device)
    net.eval()
    criterion = nn.CrossEntropyLoss()
    corrects, total_loss, tot = 0, 0.0, 0
    all_preds, all_labels = [], []

    with torch.no_grad():
        for images, labels in testloader:
            images, labels = images.to(device), labels.to(device)
            outputs = net(images)
            loss = criterion(outputs, labels)

            preds = torch.argmax(outputs, dim=1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

            corrects += torch.sum(preds == labels).item()
            total_loss += loss.item() * images.size(0)
            tot += images.size(0)

    total_loss /= tot
    accuracy = corrects / tot

    cm = confusion_matrix(all_labels, all_preds)
    TP = np.diag(cm)
    FP = np.sum(cm, axis=0) - TP
    FN = np.sum(cm, axis=1) - TP
    TN = np.sum(cm) - (TP + FP + FN)

    precision = precision_score(all_labels, all_preds, average='macro', zero_division=0)
    recall = recall_score(all_labels, all_preds, average='macro', zero_division=0)
    f1 = f1_score(all_labels, all_preds, average='macro', zero_division=0)

    return total_loss, accuracy, precision, recall, f1, TP.sum(), FP.sum(), FN.sum(), TN.sum()


def data_processing(df, NUM_FEATURES):
   y_train = df['Label']
   flow_id = df['flow_id']

   df = df/255

   X_train = df.drop(['Label', 'flow_id'], axis=1)
   X_train = X_train.to_numpy()

   X_train = X_train.reshape(-1, 20, NUM_FEATURES)
   y_train = y_train.to_numpy()

   y_train = y_train.reshape(-1,20)[:,-1]
   return torch.from_numpy(X_train), torch.from_numpy(y_train)

In [ ]:
from torch import nn

class BN_CNN(nn.Module):
    def __init__(self, in_channel, num_classes=3):
        super().__init__()
        self.conv1 = nn.Conv2d(
            in_channels=in_channel,
            out_channels=128,
            kernel_size=5,
            padding='same')

        self.conv2 = nn.Conv2d(
            in_channels=128,
            out_channels=64,
            kernel_size=5,
            padding='same')

        self.conv3 = nn.Conv2d(
            in_channels=64,
            out_channels=64,
            kernel_size=3,
            padding='same')

        self.conv4 = nn.Conv2d(
            in_channels=64,
            out_channels=32,
            kernel_size=3,
            padding='same')

        self.conv5 = nn.Conv2d(
            in_channels=32,
            out_channels=32,
            kernel_size=3,
            padding='same')

        self.conv6 = nn.Conv2d(
            in_channels=32,
            out_channels=16,
            kernel_size=3,
            padding='same')

        self.pool = nn.MaxPool2d(kernel_size=2)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.1)
        self.bn1 = nn.BatchNorm2d(num_features=64)
        self.bn2 = nn.BatchNorm2d(num_features=32)
        self.bn3 = nn.BatchNorm2d(num_features=16)
      
        self.lin1 = nn.Linear(16 * 2 * 32, 256)
        self.classification = nn.Linear(256, out_features=num_classes)

    def forward(self, X):
        X = self.pool(self.bn1(self.relu(self.conv2(self.relu(self.conv1(X))))))
        X = self.pool(self.bn2(self.relu(self.conv4(self.relu(self.conv3(X))))))
        X = self.pool(self.bn3(self.relu(self.conv6(self.relu(self.conv5(X))))))

        X = torch.flatten(X, start_dim=1) 
        X = self.lin1(X)
        X = self.relu(X)
        X = self.dropout(X)
        X = self.classification(X)

        return X

In [ ]:
from torch import nn

class CNN(nn.Module):
    def __init__(self, in_channel, num_classes=3):
        super().__init__()
        self.conv1 = nn.Conv2d(
            in_channels=in_channel,
            out_channels=128,
            kernel_size=5,
            padding='same')

        self.conv2 = nn.Conv2d(
            in_channels=128,
            out_channels=64,
            kernel_size=5,
            padding='same')

        self.conv3 = nn.Conv2d(
            in_channels=64,
            out_channels=64,
            kernel_size=3,
            padding='same')

        self.conv4 = nn.Conv2d(
            in_channels=64,
            out_channels=32,
            kernel_size=3,
            padding='same')

        self.conv5 = nn.Conv2d(
            in_channels=32,
            out_channels=32,
            kernel_size=3,
            padding='same')

        self.conv6 = nn.Conv2d(
            in_channels=32,
            out_channels=16,
            kernel_size=3,
            padding='same')

        self.conv7 = nn.Conv2d(
            in_channels=16,
            out_channels=16,
            kernel_size=3,
            padding='same')

        self.conv8 = nn.Conv2d(
            in_channels=16,
            out_channels=8,
            kernel_size=3,
            padding='same')

        self.pool = nn.MaxPool2d(kernel_size=2)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.1)

        self.lin1 = nn.Linear(16 * 2 * 32, 256)
        self.classification = nn.Linear(256, out_features=num_classes)

    def forward(self, X):
        X = self.pool(self.relu(self.conv2(self.relu(self.conv1(X)))))
        X = self.pool(self.relu(self.conv4(self.relu(self.conv3(X)))))
        X = self.pool(self.relu(self.conv6(self.relu(self.conv5(X)))))

        X = torch.flatten(X, start_dim=1)
        X = self.lin1(X)
        X = self.relu(X)
        X = self.dropout(X)
        X = self.classification(X)

        return X

In [10]:
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, input_data, targets, transform=None):
        # Ép kiểu float32 cho mỗi tensor
        self.data = [input_data[i].unsqueeze(0).float() for i in range(input_data.size(0))]
        self.targets = targets
        self.classes = torch.unique(targets).tolist()

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], self.targets[idx]


In [ ]:
c1_data = pd.read_feather('/home/bkcs/Documents/Fair/FL2/data/Domain 1.feather')
c2_data = pd.read_feather('/home/bkcs/Documents/Fair/FL2/data/Domain 2.feather')
c3_data = pd.read_feather('/home/bkcs/Documents/Fair/FL2/data/Domain 3.feather')

data_full = [c1_data, c2_data , c3_data]
df_full = pd.concat(data_full)
df_full

,flow_id,0,1,2,3,4,5,6,7,8,...,247,248,249,250,251,252,253,254,255,Label
7,3418,23,3,3,1,58,225,139,5,233,...,77,188,75,60,105,185,244,201,71,0
6,3418,23,3,3,0,165,58,55,16,196,...,0,0,0,0,0,0,0,0,0,0
5,3418,20,3,3,0,1,1,23,3,3,...,0,0,0,0,0,0,0,0,0,0
4,3418,23,3,3,0,47,122,79,32,128,...,0,0,0,0,0,0,0,0,0,0
3,3418,23,3,3,0,154,85,85,156,61,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
124935,212962,144,239,122,158,85,81,80,164,12,...,0,0,0,0,0,0,0,0,0,2
124936,212962,144,111,122,176,85,81,148,36,12,...,0,0,0,0,0,0,0,0,0,2
124937,212962,144,111,122,166,85,81,110,164,12,...,0,0,0,0,0,0,0,0,0,2
124938,212962,144,111,122,177,85,81,151,228,12,...,0,0,0,0,0,0,0,0,0,2


In [41]:
from torch.utils.data import DataLoader
import numpy as np
import random

def set_seed(seed_value):
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed_value)
    print(f"Seeds set to {seed_value}")

BATCH_SIZE = 32
RANDOM_STATE = 42
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

set_seed(RANDOM_STATE)

all_data = data_processing(df_full, 256)
full_dataset = CustomDataset(all_data[0], all_data[1])

trainset, testset = train_test_split(full_dataset, test_size=0.2, random_state=RANDOM_STATE)

trainloader = DataLoader(trainset, batch_size=BATCH_SIZE, num_workers=4)
testloader = DataLoader(testset, batch_size=BATCH_SIZE, num_workers=4)

Seeds set to 42


In [45]:

def training_loop(net, trainloader, testloader, device, epochs=10, proximal_mu=None):
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(net.parameters(), lr=0.003)

    for epoch in range(epochs):
        train_loss, train_acc, train_prec, train_recall, train_f1, TP, FP, FN, TN = train(
            net, trainloader, criterion, optimizer, device, proximal_mu)

        test_loss, test_acc, test_prec, test_recall, test_f1, TP_t, FP_t, FN_t, TN_t = test(
            net, testloader, device)

        print(f"Epoch {epoch+1}/{epochs}")
        print(f"Train Loss: {train_loss:.4f}, Acc: {train_acc:.4f}, Prec: {train_prec:.4f}, Recall: {train_recall:.4f}, F1: {train_f1:.4f}")
        print(f"Train TP: {TP}, FP: {FP}, FN: {FN}, TN: {TN}")
        print(f"Test  Loss: {test_loss:.4f}, Acc: {test_acc:.4f}, Prec: {test_prec:.4f}, Recall: {test_recall:.4f}, F1: {test_f1:.4f}")
        print(f"Test  TP: {TP_t}, FP: {FP_t}, FN: {FN_t}, TN: {TN_t}")
        print("-" * 80)

    
model = BN_CNN(1,3).to(DEVICE)

training_loop(model, trainloader, testloader, DEVICE, 10)

Epoch 1/10
Train Loss: 0.3660, Acc: 0.8557, Prec: 0.8476, Recall: 0.8422, F1: 0.8435
Train TP: 15735, FP: 2653, FN: 2653, TN: 34123
Test  Loss: 0.2083, Acc: 0.9234, Prec: 0.9193, Recall: 0.9194, F1: 0.9193
Test  TP: 4246, FP: 352, FN: 352, TN: 8844
--------------------------------------------------------------------------------
Epoch 2/10
Train Loss: 0.1899, Acc: 0.9295, Prec: 0.9249, Recall: 0.9227, F1: 0.9236
Train TP: 17091, FP: 1297, FN: 1297, TN: 35479
Test  Loss: 0.2159, Acc: 0.9200, Prec: 0.9179, Recall: 0.9216, F1: 0.9171
Test  TP: 4230, FP: 368, FN: 368, TN: 8828
--------------------------------------------------------------------------------
Epoch 3/10
Train Loss: 0.1459, Acc: 0.9484, Prec: 0.9456, Recall: 0.9432, F1: 0.9442
Train TP: 17440, FP: 948, FN: 948, TN: 35828
Test  Loss: 0.2109, Acc: 0.9291, Prec: 0.9282, Recall: 0.9318, F1: 0.9269
Test  TP: 4272, FP: 326, FN: 326, TN: 8870
--------------------------------------------------------------------------------
Epoch 4/10
T